In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Baseline_MUSS_corrupt/muss
!pip install -e . 
# !pip install -U spacy ==2.0.0
!python -m spacy download en_core_web_md



Sat Sep 17 22:40:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    51W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:

from muss.mining.training import get_bart_kwargs, get_score_rows
from muss.resources.prepare import prepare_wikilarge_detokenized, prepare_asset
from muss.resources.datasets import create_smaller_dataset

from muss.fairseq.main import fairseq_train_and_evaluate_with_parametrization
from muss.mining.training import get_score_rows #get_bart_kwargs

#########################
import re
import shutil
import os 

from cachier import cachier
from easse.cli import evaluate_system_output
from easse.utils.constants import TEST_SETS_PATHS
import torch
from tqdm import tqdm

from muss.resources.paths import get_data_filepath, MODELS_DIR, get_dataset_dir
from muss.utils.helpers import add_dicts, args_str_to_dict
from muss.utils.resources import download_and_extract
from muss.preprocessors import GPT2BPEPreprocessor
from muss.preprocessing import apply_line_function_to_file
from muss.fairseq.main import get_language_from_dataset
from muss.text import truncate

from muss.simplify import simplify_sentences
from muss.utils.helpers import *

from muss.simplifiers import get_fairseq_simplifier, get_preprocessed_simplifier
from muss.preprocessors import get_preprocessors, get_preprocessor_by_name

from muss.mining.training import prepare_bart_model

In [3]:

# This dataset should exist in resources/datasets/ and contain the following files:
# train.complex, train.simple, valid.complex, valid.simple, test.complex, test.simple
def get_evaluate_kwargs(language, phase='valid'):
    return {
        ('en', 'valid'): {'test_set': 'asset_valid'},
        ('en', 'test'): {'test_set': 'asset_test'},
        ('fr', 'valid'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('alector', 'valid', 'complex'),
            'refs_sents_paths': [get_data_filepath('alector', 'valid', 'simple')],
        },
        ('fr', 'test'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('alector', 'test', 'complex'),
            'refs_sents_paths': [get_data_filepath('alector', 'test', 'simple')],
        },
        ('es', 'valid'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('simplext_corpus', 'valid', 'complex'),
            'refs_sents_paths': [get_data_filepath('simplext_corpus', 'valid', 'simple')],
        },
        ('es', 'test'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('simplext_corpus', 'test', 'complex'),
            'refs_sents_paths': [get_data_filepath('simplext_corpus', 'test', 'simple')],
        },
    }[(language, phase)]

evaluate_kwargs=get_evaluate_kwargs('en')


def get_predict_files(language):
    return {
        'en': [get_data_filepath('asset', 'valid', 'complex'), get_data_filepath('asset', 'test', 'complex')],
        'fr': [get_data_filepath('alector', 'valid', 'complex'), get_data_filepath('alector', 'test', 'complex')],
        'es': [
            get_data_filepath('simplext_corpus', 'valid', 'complex'),
            get_data_filepath('simplext_corpus', 'test', 'complex'),
        ],
    }[language]

def get_access_preprocessors_kwargs(language, use_short_name=False):
    return {
        'LengthRatioPreprocessor': {'target_ratio': 0.8, 'use_short_name': use_short_name},
        'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.8, 'use_short_name': use_short_name},
        'WordRankRatioPreprocessor': {'target_ratio': 0.8, 'language': language, 'use_short_name': use_short_name},
        'DependencyTreeDepthRatioPreprocessor': {
            'target_ratio': 0.8,
            'language': language,
            'use_short_name': use_short_name,
        },
    }

# def get_bart_kwargs(dataset, language, use_access, use_short_name=False, bart_model='bart.large'):
#     assert language == 'en'

#     # bart_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/models/muss_en_wikilarge_mined/model.pt' 
#     bart_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/models/bart.large/model.pt' 

#     arch = {
#         'bart.base': 'bart_base',
#         'bart.large': 'bart_large',
#         'bart.large.cnn': 'bart_large',
#     }[bart_model]
#     kwargs = {
#         'dataset': dataset,
#         'metrics_coefs': [0, 1, 0],
#         'parametrization_budget': 128,
#         'predict_files': get_predict_files(language), # used in fairseq_evaluate_and_save
#         'preprocessors_kwargs': {
#             'GPT2BPEPreprocessor': {},
#         },
#         'preprocess_kwargs': {'dict_path': GPT2BPEPreprocessor().dict_path},
#         'train_kwargs': {
#             'ngpus': 1,
#             'arch': arch,
#             'restore_file': bart_path,
#             'max_tokens': 4096,
#             'lr': 3e-05,
#             'warmup_updates': 500, #500
#             'truncate_source': True,
#             'layernorm_embedding': True,
#             'share_all_embeddings': True,
#             'share_decoder_input_output_embed': True,
#             'reset_optimizer': True,
#             'reset_dataloader': True,
#             'reset_meters': True,
#             'required_batch_size_multiple': 1,
#             'criterion': 'label_smoothed_cross_entropy',
#             'label_smoothing': 0.1,
#             'dropout': 0.1,
#             'attention_dropout': 0.1,
#             'weight_decay': 0.01,
#             'optimizer': 'adam',
#             'adam_betas': '(0.9, 0.999)',
#             'adam_eps': 1e-08,
#             'clip_norm': 0.1,
#             'lr_scheduler': 'polynomial_decay',
#             'max_update': 10000, # 20000
#             'skip_invalid_size_inputs_valid_test': True,
#             'find_unused_parameters': True,
#         },
#         'evaluate_kwargs': get_evaluate_kwargs(language),
#     }
#     if use_access:
#         kwargs['preprocessors_kwargs'] = add_dicts(
#             get_access_preprocessors_kwargs(language, use_short_name=use_short_name), kwargs['preprocessors_kwargs']
#         )
#     return kwargs



def get_bart_kwargs(dataset, language, use_access, use_short_name=False, bart_model='bart.large'):
    assert language == 'en'
    # bart_path = prepare_bart_model(bart_model) / 'model.pt'
    bart_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/models/muss_en_wikilarge_mined/model.pt'
    arch = {
        'bart.base': 'bart_base',
        'bart.large': 'bart_large',
        'bart.large.cnn': 'bart_large',
    }[bart_model]
    kwargs = {
        'dataset': dataset,
        'metrics_coefs': [0, 1, 0],
        'parametrization_budget': 128,
        'predict_files': get_predict_files(language),
        'preprocessors_kwargs': {
            'GPT2BPEPreprocessor': {},
        },
        'preprocess_kwargs': {'dict_path': GPT2BPEPreprocessor().dict_path},
        'train_kwargs': {
            'ngpus': 8,
            'arch': arch,
            'restore_file': bart_path,
            'max_tokens': 4096,
            'lr': 3e-05,
            'warmup_updates': 500,
            'truncate_source': True,
            'layernorm_embedding': True,
            'share_all_embeddings': True,
            'share_decoder_input_output_embed': True,
            'reset_optimizer': True,
            'reset_dataloader': True,
            'reset_meters': True,
            'required_batch_size_multiple': 1,
            'criterion': 'label_smoothed_cross_entropy',
            'label_smoothing': 0.1,
            'dropout': 0.1,
            'attention_dropout': 0.1,
            'weight_decay': 0.01,
            'optimizer': 'adam',
            'adam_betas': '(0.9, 0.999)',
            'adam_eps': 1e-08,
            'clip_norm': 0.1,
            'lr_scheduler': 'polynomial_decay',
            'max_update': 20000,
            'skip_invalid_size_inputs_valid_test': True,
            'find_unused_parameters': True,
        },
        'evaluate_kwargs': get_evaluate_kwargs(language),
    }
    if use_access:
        kwargs['preprocessors_kwargs'] = add_dicts(
            get_access_preprocessors_kwargs(language, use_short_name=use_short_name), kwargs['preprocessors_kwargs']
        )
    return kwargs

In [4]:
dataset = 'NE_recovery_prefix_3'
data_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/'
%ls '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/wikilarge_tag'

# read_lines(data_dir+dataset+'/test.complex')

test.complex  train.complex  valid.complex
test.simple   train.simple   valid.simple


In [5]:
# read_lines(data_dir+dataset+'/test.simple')

In [6]:
# dataset = 'NE_recovery'
kwargs = get_bart_kwargs(dataset=dataset, language='en', use_access=True)
kwargs['train_kwargs']['ngpus'] = 1  # Set this from 8 to 1 for local training
kwargs['train_kwargs']['max_tokens'] = 512  # Lower this number to prevent OOM
kwargs['train_kwargs']['warmup_updates'] = 1000
kwargs['train_kwargs']['max_update'] = 4000
kwargs['train_kwargs']['save_interval_updates'] = 500
result = fairseq_train_and_evaluate_with_parametrization(**kwargs)
print('Training done')

let the training begin
fairseq_prepare_and_train...



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/muss/utils/helpers.py", line 384, in __enter__
    sys.settrace(lambda *args, **keys: None)



exp_dir=/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1663454458528
preprocess
save_interval =  500
fairseq-train /content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/_0c7da87e7d358e6f194eab1c3b07201b/fairseq_preprocessed_complex-simple --task translation --source-lang complex --target-lang simple --save-dir /content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1663454458528/checkpoints --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 --criterion label_smoothed_cross_entropy --label-smoothing 0.1 --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 1000 --update-freq 128 --arch bart_large --dropout 0.1 --weight-decay 0.0 --clip-norm 0.1 --share-all-embeddings --no-epoch-checkpoints --save-interval 999999 --validate-interval 999999 --max-update 4000 --save-interval-updates 500 --keep-interval-updates 1 --patience 10 --batch-size 64 --seed 968 --distributed-world-size 1 --distributed-port 11647 --fp16 -

fairseq_prepare_and_train completed after 56097.92s.
22222222222222222222222222222222222222222222222222
find_best_parametrization...


/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // beam_size
[nltk_data] Downloading package punkt to 

...

report_path=/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1663454458528/easse_report.html


/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning:

__floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').

/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning:

__floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').



source_path=/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/valid.complex
pred_path=/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1663454458528/valid.pred
source_path=/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/test.complex
pred_path=/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1663454458528/test.pred
fairseq_evaluate_and_save completed after 319.26s.


/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning:

__floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').

/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning:

__floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').

100%|██████████| 128/128 [2:44:15<00:00, 77.00s/it]


FileNotFoundError: ignored

In [ ]:
exp_dir ='/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1662500425274/'


from muss.fairseq.main import find_best_parametrization
recommended_preprocessors_kwargs = find_best_parametrization(exp_dir, **kwargs)
print(f'recommended_preprocessors_kwargs={recommended_preprocessors_kwargs}')
kwargs['preprocessor_kwargs'] = recommended_preprocessors_kwargs

/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // beam_size
100%|██████████| 128/128 [2:49:49<00:00, 

recommended_preprocessors_kwargs={'LengthRatioPreprocessor': {'target_ratio': 0.8421908748934228, 'use_short_name': False}, 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.8480454493132732, 'use_short_name': False}, 'WordRankRatioPreprocessor': {'target_ratio': 0.8832210383250984, 'language': 'en', 'use_short_name': False}, 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 0.3388928469969324, 'language': 'en', 'use_short_name': False}, 'GPT2BPEPreprocessor': {}}


In [ ]:
print(f'recommended_preprocessors_kwargs={recommended_preprocessors_kwargs}')

In [ ]:
read_lines('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/0913_ABCD_NER_wikilarge/train.simple')

['There is some proof that Austen continued to work on these pieces later in life. Her nephew and niece, James Edward and Anna Austen, may have made further additions to her work in around 1814.',
 "Mandaean scholar Säve-Söderberg showed that Mani's Psalms of Thomas were closely related to Mandaean texts.",
 'When Demeter went to the Underworld to rescue her Persephone, Hades forced Persephone to eat the pomegranate. After she ate this fruit it was supposed to keep her in the underworld with Hades so she would be forced to marry him.',
 "The city's main newspaper is the Tribune de Genève, a daily newspaper founded on 1 February 1879 by James T. Bates, with a readership of about 187,000.",
 'A scythe (, from Old English siðeOxford English Dictionary 1933.)',
 'Some of the plays and comic puppet theater from around the 16th century make up their own versions of the story. They often show Faust as a figure of vulgar fun.',
 "In September 1869, O'Reilly escaped with help from an American s

# Load and evaluate 

In [ ]:
MODEL_DIR = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/'

model_dir_dict = {}
def add_item_to_dict(**kwargs):

  id = len(model_dir_dict)
  model_dir_dict[id]={}

  for key, value in kwargs.items():
    model_dir_dict[id][key]=value
  print('added:',model_dir_dict[id])

add_item_to_dict(model_id=len(model_dir_dict),exp_dir= MODEL_DIR+'local_1661468071184/',model_name='NE_recovery_model',recommended_preprocessors_kwargs={'LengthRatioPreprocessor': {'target_ratio': 1.0561701019588032, 'use_short_name': False}, 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.9757367588571789, 'use_short_name': False}, 'WordRankRatioPreprocessor': {'target_ratio': 0.7395441030797956, 'language': 'en', 'use_short_name': False}, 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 0.5234261949274427, 'language': 'en', 'use_short_name': False}, 'GPT2BPEPreprocessor': {}})
test_data_dir_list,test_data_list = ['/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/NE_recovery/test.complex'],['ne.valid.complex.pred']
model_dir_dictx = model_dir_dict

model_dir_dictx[0]

added: {'model_id': 0, 'exp_dir': '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1661468071184/', 'model_name': 'NE_recovery_model', 'recommended_preprocessors_kwargs': {'LengthRatioPreprocessor': {'target_ratio': 1.0561701019588032, 'use_short_name': False}, 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.9757367588571789, 'use_short_name': False}, 'WordRankRatioPreprocessor': {'target_ratio': 0.7395441030797956, 'language': 'en', 'use_short_name': False}, 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 0.5234261949274427, 'language': 'en', 'use_short_name': False}, 'GPT2BPEPreprocessor': {}}}


{'model_id': 0,
 'exp_dir': '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1661468071184/',
 'model_name': 'NE_recovery_model',
 'recommended_preprocessors_kwargs': {'LengthRatioPreprocessor': {'target_ratio': 1.0561701019588032,
   'use_short_name': False},
  'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.9757367588571789,
   'use_short_name': False},
  'WordRankRatioPreprocessor': {'target_ratio': 0.7395441030797956,
   'language': 'en',
   'use_short_name': False},
  'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 0.5234261949274427,
   'language': 'en',
   'use_short_name': False},
  'GPT2BPEPreprocessor': {}}}

In [ ]:
def generate_output(chosen_id_list,data_dir_list,data_list,model_dir_dict,sample=False,add_to_dir_name=None,muss_output=False,NE_output=False,CERF_output=False,NE_CERF_output=False,hypothesis_num=None,beam=None,sampling=False):

  # store created dir
  created_dir = []
  output_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/'
  for index in chosen_id_list:
    
    # get model unique local id
    exp_dir = model_dir_dict[index]['exp_dir']
    print('| model_name is',model_dir_dict[index]['model_name'])
    print('| exp_dir is',exp_dir)

    model_name = 'model_'+ str(index) + '_'+ exp_dir.split('/')[-2]

    os.makedirs(output_dir+model_name, exist_ok=True)
    print('| output_dir',output_dir)
    language = 'en'

    # get recommended_preprocessors_kwargs
    if 'recommended_preprocessors_kwargs' in model_dir_dict[index].keys() :
      preprocessors_kwargs = model_dir_dict[index]['recommended_preprocessors_kwargs']
      print('| use provided kwargs')
    else:
      # use muss_mined_wiki preprocessors
      preprocessors_kwargs = {
            'LengthRatioPreprocessor': {'target_ratio': 0.9, 'use_short_name': False},
            'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.65, 'use_short_name': False},
            'WordRankRatioPreprocessor': {'target_ratio': 0.75, 'language': language, 'use_short_name': False},
            'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 0.4, 'language': language, 'use_short_name': False},
        }
      preprocessors_kwargs['GPT2BPEPreprocessor'] = {}

    print('| use preprocessors_kwargs',preprocessors_kwargs)
    preprocessors = get_preprocessors(preprocessors_kwargs)

    if hypothesis_num == None:
      hypothesis_num = 1
    if beam == None:
      beam=5

    generate_kwargs = {'hypothesis_num':hypothesis_num,
                       'beam':beam,
                       'sampling':sampling}

    simplifier = get_fairseq_simplifier(exp_dir, **generate_kwargs)
    simplifier = get_preprocessed_simplifier(simplifier, preprocessors=preprocessors)

    #########################################
    # simplify sample sentences 

    ############################## I need to change the directory 
    if sample:
      if muss_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_no_token.en'
      elif NE_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_NE_token.en'
      elif CERF_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_ABCD_token.en'
      elif NE_CERF_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_NE_ABCD_token.en'

      pred_path = simplifier(complex_file_dir)

      for i in range(len(read_lines(complex_file_dir))):
        print('original:\n',read_lines(complex_file_dir)[i])
        print('simplified:\n',read_lines(pred_path)[i])
        print('----------------------------------------------------')

      return read_lines(complex_file_dir),read_lines(pred_path)
    ###########################################

    # create dir for output
    subfolders = os.listdir(output_dir + model_name)
    num_subfolder = len(subfolders)
    new_folder_dir = output_dir + model_name + '/' +str(num_subfolder).zfill(2) +'/'
    os.mkdir(new_folder_dir)
    created_dir.append(new_folder_dir)

     # simplify asset dataset
    for index, file_dir in enumerate(data_dir_list):
      
      info_file_name = new_folder_dir + 'info.txt'
      with open(info_file_name, 'w') as f:
        f.write("generate_kwargs %s\n" % generate_kwargs)
      f.close()

      filename = new_folder_dir + data_list[index]
      print('filename;', filename)
      pred_path = simplifier(file_dir)

      with open(filename, 'w') as f:
          for item in read_lines(pred_path):
              f.write("%s\n" % item)

  print('created_dir',created_dir)
  return created_dir

In [ ]:
token_ori_2 = generate_output([0],test_data_dir_list,test_data_list, model_dir_dictx)

| model_name is NE_recovery_model
| exp_dir is /content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1661468071184/
| output_dir /content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/
| use provided kwargs
| use preprocessors_kwargs {'LengthRatioPreprocessor': {'target_ratio': 1.0561701019588032, 'use_short_name': False}, 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.9757367588571789, 'use_short_name': False}, 'WordRankRatioPreprocessor': {'target_ratio': 0.7395441030797956, 'language': 'en', 'use_short_name': False}, 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 0.5234261949274427, 'language': 'en', 'use_short_name': False}, 'GPT2BPEPreprocessor': {}}
filename; /content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/model_0_local_1661468071184/13/ne.valid.complex.pred
created_dir ['/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/model_0_local_1661468071184/13/']


### Show some result

In [ ]:
comp_sentence, simp_sentence = show_simplify(test_data_dir_list[0],token_ori_2[0]+test_data_list[0])

simplified:

-------------------------------------------------------------------------------- 
 complex:  <NEXT_NE> judicial <NEXT_DIFFICULT_WORD> Putrajaya. <SENT_START> The administrative and financial functions of the government were shifted from Kuala Lumpur to lumpur
simple: The administrative and financial functions of the government were shifted from Kuala to lumprauraya.
-------------------------------------------------------------------------------- 
 complex:  <NEXT_NE> Originally <NEXT_DIFFICULT_WORD> disc <NEXT_DIFFICULT_WORD> edition <SENT_START> also released as a bonus single on the deluxe form of B 'Day
simple: Originally released as a bonus disc on the deluxe edition of B 'Day
-------------------------------------------------------------------------------- 
 complex:  <NEXT_NE> JVM <NEXT_DIFFICULT_WORD> component <NEXT_DIFFICULT_WORD> Platform. <SENT_START> The language is a crucial part of the Java system
simple: The language is a crucial component of the Java compone

### Analyze recovered entity

In [ ]:
### Compute hardword word retention rate 
def find_occurence(NE,simple_sentence):
  NE_simp = [i.lower() for i in NE['NE_simple']]
  NE_diff = [i.lower() for i in NE['NE_difficult']]
  simple_sentence = simple_sentence.lower()
  intersect_words = []

  count_simple = 0
  count_difficult = 0
  # print(NE_diff)
  # print(NE)
  if NE_simp[0] != '' or NE_diff:
    # print(NE_simp[0] == '')
  # print(sentence,test_complex_NER_spacy[index])
    if NE_simp[0] != '':
      for text in NE_simp:
        if text.lower() in simple_sentence:
          intersect_words.append(text)
          count_simple +=1
    
    if NE_diff:
      for text in NE_diff:
        if text.lower() in simple_sentence:
          intersect_words.append(text)
          count_difficult +=1
          # print('diff words found in sen')

    return count_simple, count_difficult
  else:
    # print('empty NE')
    return 0,0
    
def word_retention_rate(comp_sentence,simp_sentence):
  total_simp = 0
  total_diff = 0
  counter_simp = 0
  counter_diff = 0

  for i,sentence in enumerate(simp_sentence):
    NE_temp1 = NE_extraction(comp_sentence[i])[0]

    count_simp, count_diff = find_occurence(NE_temp1,sentence)

    total_simp += len(NE_temp1['NE_simple'])
    total_diff += len(NE_temp1['NE_difficult'])

    counter_simp += count_simp
    counter_diff += count_diff
    # print(count_simp, count_diff)
  print('simple_NE retention rate',counter_simp/total_simp)
  print('difficult_NE retention rate',counter_diff/total_diff)
  print('total_NE retention rate', (counter_simp+counter_diff)/(total_simp+total_diff))
  return counter_simp/total_simp,counter_diff/total_diff, (counter_simp+counter_diff)/(total_simp+total_diff)

In [ ]:
word_retention_rate(comp_sentence, simp_sentence)

simple_NE retention rate 0.44889779559118237
difficult_NE retention rate 0.34823529411764703
total_NE retention rate 0.4025974025974026


(0.44889779559118237, 0.34823529411764703, 0.4025974025974026)

# Full pipeline analysis - Simplify - recovery

In [ ]:
MODEL_DIR = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/'
Temp_data_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/'
Temp_data_name = 'process_sentence1'
model_dir_dict = {}
def add_item_to_dict(**kwargs):

  id = len(model_dir_dict)
  model_dir_dict[id]={}

  for key, value in kwargs.items():
    model_dir_dict[id][key]=value
  print('added:',model_dir_dict[id])

add_item_to_dict(model_id=len(model_dir_dict),exp_dir= MODEL_DIR+'local_1660919962534/',model_name='WIKI_Simplify')
test_data_dir_list,test_data_list = ['/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/0913_ABCD_NER_wikilarge/valid.complex'],['valid.diff.complex.pred']
model_dir_dictx = model_dir_dict
model_dir_dictx[0]

added: {'model_id': 0, 'exp_dir': '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1660919962534/', 'model_name': 'WIKI_Simplify'}


{'model_id': 0,
 'exp_dir': '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1660919962534/',
 'model_name': 'WIKI_Simplify'}

In [ ]:
token_ori_1 = generate_output([0],test_data_dir_list,test_data_list, model_dir_dictx)

| model_name is NE_recovery
| exp_dir is /content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1661468071184/
| output_dir /content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/
| use provided kwargs
| use preprocessors_kwargs {'LengthRatioPreprocessor': {'target_ratio': 1.0561701019588032, 'use_short_name': False}, 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.9757367588571789, 'use_short_name': False}, 'WordRankRatioPreprocessor': {'target_ratio': 0.7395441030797956, 'language': 'en', 'use_short_name': False}, 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 0.5234261949274427, 'language': 'en', 'use_short_name': False}, 'GPT2BPEPreprocessor': {}}
filename; /content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/model_0_local_1661468071184/15/valid.diff.complex.pred.1


/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // beam_size


created_dir ['/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/model_0_local_1661468071184/15/']


In [ ]:
comp_sentence1, simp_sentence1 = show_simplify(test_data_dir_list[0],token_ori_1[0]+test_data_list[0])

simplified:

-------------------------------------------------------------------------------- 
 complex:  <NEXT_NE> the European Charter for Regional or Minority Languages <NEXT_NE> around 40,000 <NEXT_NE> Upper Sorbian <NEXT_NE> Germany <NEXT_NE> Saxony <NEXT_DIFFICULT_WORD> ica <SENT_START> The Upper Sorbian (Hornja) is a minority language spoken in Germany. It is spoken by the European Charter for Regional or Minority Languages (Hornja Å uÅ uÅ uÅ uÅ uÅ 3\/4 ica) in Saxony. It is the capital of Saxony (Hornja), which is the largest city in the European Charter for Regional or Minority Languages (Hornja).
simple: The Upper Fony for the European town in the European town (Hica) is spoken by the European language or spoken in Germany. It is spoken by the European number of the European football (Hony in Germany.
-------------------------------------------------------------------------------- 
 complex:  <NEXT_NE>  <NEXT_DIFFICULT_WORD> evolved <SENT_START> A shoe is an item of footwear 

In [ ]:
word_retention_rate(comp_sentence1, simp_sentence1)

simple_NE retention rate 0.7618243243243243
difficult_NE retention rate 0.5127659574468085
total_NE retention rate 0.6516007532956686


(0.7618243243243243, 0.5127659574468085, 0.6516007532956686)

In [ ]:
def reconstruct_simplified_data(complex_sentence, simple_sentence):
    def list_duplicates_of(seq,item):
      start_at = -1
      locs = []
      while True:
          try:
              loc = seq.index(item,start_at+1)
          except ValueError:
              break
          else:
              locs.append(loc)
              start_at = loc
      return locs
    control_tokens = ["<NEXT_NE> ","<NEXT_DIFFICULT_WORD>","<SENT_START>"]
    indice_of_start = list_duplicates_of(complex_sentence,control_tokens[2])
    
    sentence = complex_sentence[0:indice_of_start[0]+len(control_tokens[2])+1] + simple_sentence
    return sentence


In [ ]:
sentences = []
for i, sen in enumerate(comp_sentence):
  sentence = reconstruct_simplified_data(sen,simp_sentence[i])
  # print(type(sentence))
  # sentence = 
  sentences.append(sentence)
print(sentences)


[' <NEXT_NE> the European Charter for Regional or Minority Languages <NEXT_NE> around 40,000 <NEXT_NE> Upper Sorbian <NEXT_NE> Germany <NEXT_NE> Saxony <NEXT_DIFFICULT_WORD> ica <SENT_START> The Upper Sorbian (Hornja) is a minority language spoken in Germany. It is spoken by the European Charter for Regional or Minority Languages (Hornja Å uÅ uÅ uÅ uÅ uÅ 3\\/4 ica) in Saxony. It is the capital of Saxony (Hornja), which is the largest city in the European Charter for Regional or Minority Languages (Hornja).', ' <NEXT_NE>  <NEXT_DIFFICULT_WORD> evolved <SENT_START> A shoe is an item of footwear that is used to protect the human foot and to protect the human foot and later, as an item of decoration, as an item of decoration in itself.', ' <NEXT_NE> the eastern Pacific <NEXT_NE> November 30, 1995 <NEXT_NE> May 15, 1995 <NEXT_NE> June 1, 1995 <NEXT_NE> season <NEXT_DIFFICULT_WORD> tropical <NEXT_DIFFICULT_WORD> cyclones <NEXT_DIFFICULT_WORD> affected <SENT_START> On June 1, 1995, the season

In [ ]:
with open(Temp_data_dir+Temp_data_name,'w') as f:
  # print(type(sentence))
  for sen in sentences:
    f.write("{}\n".format(sen))
f.close()

In [ ]:
# comp_sentence = show_simplify(Temp_data_dir+Temp_data_name)
# comp_sentence

In [ ]:
model_dir_dict = {}
add_item_to_dict(model_id=len(model_dir_dict),exp_dir= MODEL_DIR+'local_1661468071184/',model_name='NE_recovery')
#recommended_preprocessors_kwargs={'LengthRatioPreprocessor': {'target_ratio': 1.0561701019588032, 'use_short_name': False}, 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.9757367588571789, 'use_short_name': False}, 'WordRankRatioPreprocessor': {'target_ratio': 0.7395441030797956, 'language': 'en', 'use_short_name': False}, 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 0.5234261949274427, 'language': 'en', 'use_short_name': False}, 'GPT2BPEPreprocessor': {}}
test_data_dir_list,test_data_list = [Temp_data_dir+Temp_data_name],['valid.diff.complex.pred.1']
model_dir_dictx = model_dir_dict
model_dir_dictx[0]
comp_sentence = show_simplify(test_data_dir_list[0])
# comp_sentence[0]

added: {'model_id': 0, 'exp_dir': '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1661468071184/', 'model_name': 'NE_recovery'}


In [ ]:
token_ori_2 = generate_output([0],test_data_dir_list,test_data_list, model_dir_dictx)
comp_sentence2, simp_sentence2 = show_simplify(test_data_dir_list[0],token_ori_2[0]+test_data_list[0])

| model_name is NE_recovery
| exp_dir is /content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1661468071184/
| output_dir /content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/
| use preprocessors_kwargs {'LengthRatioPreprocessor': {'target_ratio': 0.9, 'use_short_name': False}, 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.65, 'use_short_name': False}, 'WordRankRatioPreprocessor': {'target_ratio': 0.75, 'language': 'en', 'use_short_name': False}, 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 0.4, 'language': 'en', 'use_short_name': False}, 'GPT2BPEPreprocessor': {}}
filename; /content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/model_0_local_1661468071184/17/valid.diff.complex.pred.1


/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // beam_size


created_dir ['/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/model_0_local_1661468071184/17/']
simplified:

-------------------------------------------------------------------------------- 
 complex:  <NEXT_NE> the European Charter for Regional or Minority Languages <NEXT_NE> around 40,000 <NEXT_NE> Upper Sorbian <NEXT_NE> Germany <NEXT_NE> Saxony <NEXT_DIFFICULT_WORD> ica <SENT_START> The Upper Sorbian (Hornja) is a minority language spoken in Germany. It is spoken by the European Charter for Regional or Minority Languages (Hornja Å uÅ uÅ uÅ uÅ uÅ 3\/4 ica) in Saxony. It is the capital of Saxony (Hornja), which is the largest city in the European Charter for Regional or Minority Languages (Hornja).
simple: The Upper Fony for the European town in the European town (Hica) is a number for the European language or spoken in Germany. It is spoken by the European number of Germany (Hony in Germany.
-------------------------------------------------------------------------------- 
 

In [ ]:
word_retention_rate(comp_sentence2, simp_sentence2)

simple_NE retention rate 0.34628378378378377
difficult_NE retention rate 0.3202127659574468
total_NE retention rate 0.3347457627118644


(0.34628378378378377, 0.3202127659574468, 0.3347457627118644)

In [ ]:
simp_sentence[0]

'The Upper Fony for the European town in the European town (Hica) is a number for the European language or spoken in Germany. It is spoken by the European number of Germany (Hony in Germany.'

In [ ]:
for i,j in enumerate(simp_sentence1):
  # print(j)
  # print(simp_sentence2[i])
  if j == simp_sentence2[i]:
    print(True)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
